<a href="https://colab.research.google.com/github/khodid/2020_SAI_MONING2/blob/master/CNN01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 모두의 딥러닝 시즌 2 - CNN



CNN은 데이터와 필터 간의 2차원 Convolution 연산을 하는 뉴런을 엮어 만드는 Neural Network다.



우리가 기존에 딥뉴럴네트워크에서 'nn.Linear'로 선형 결합을 했던 것을, CNN에선 
'torch.nn.Conv2d()' 오브젝트를 사용한다.

**torch.nn.Conv2d의 매개변수**는 다음과 같다.
- in_channels : input channel의 수
- out_channels : output channel의 수
- kernel\_size : 연산하는 2D data의 크기를 넣어준다. kernel\_size=N 으로 입력하면 NxN 크기를 나타내고, kernel\_size = (N,M)처럼 넣어 주면 NxM 크기를 나타내는 것이다,
- stride : filer를 이동시키는 간격이다.
- padding : 가장자리에 N 줄씩 0 넣고 추가해준다.

**pooling**이라는 레이어도 끼워 넣을 수 있는데, 이건 한 2D 배열에서 일정 구간마다 잘라서 하나로 나타내는 일종의 압축 기법이다.

최댓값을 가져가는 Max Pooling, 평균값을 가져가는 Mean Pooling(Average pooling)이 있다.

In [0]:
# Library

import torch
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms

import torch.nn.init

In [0]:
# GPU 설정

device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(777)
if device == 'cuda':
  torch.cuda.manual_seed_all(777)

In [0]:
# Parameter 작성 - 나중에 써도 상관 없음.

learning_rate = 0.001
batch_size = 100
training_epochs = 15

In [0]:
# MNIST dataset load
mnist_train = dsets.MNIST(root = 'MNIST_data/', train = True, transform = transforms.ToTensor(), download = True)
mnist_test  = dsets.MNIST(root = 'MNIST_data/', train = False, transform = transforms.ToTensor(), download  = True)

In [0]:
# Data Loader

data_loader = torch.utils.data.DataLoader(dataset = mnist_train, batch_size = batch_size, shuffle = True, drop_last = True)

In [0]:
# Model 정의하기

class CNN(nn.Module):
  def __init__(self):
    super(CNN, self).__init__()
    self.layer1 = nn.Sequential(
        nn.Conv2d(1, 32, kernel_size = 3, stride = 1, padding = 1),
        nn.ReLU(),
        nn.MaxPool2d(2)
    )
    self.layer2 = nn.Sequential(
        nn.Conv2d(32, 64, kernel_size = 3, stride = 1, padding = 1),
        nn.ReLU(),
        nn.MaxPool2d(2)
    )
    self.fc = nn.Linear(7*7*64, 10, bias = True ) # 7x7x64는 채널이 64 채널에 7x7인 output이 나오기 때문.
    torch.nn.init.xavier_uniform_(self.fc.weight)

  def forward(self, x):
    out = self.layer1(x)
    out = self.layer2(out)
    out = out.view(out.size(0), -1)
    out = self.fc(out)
    return out

여기서 conv2d와 Maxpool2d 레이어를 거치며 변하는 shape에 대한 건 [모딥 깃헙 소스](github/deeplearningzerotoall/PyTorch/blob/master/lab-10_2_mnist_deep_cnn.ipynb)에 잘 설명되어있다.

솔직히 좀 어려운데... 익숙해지면 이해가 되지 않을까 생각한다.


In [0]:
# 모델 선언
model = CNN().to(device)   # device는 cuda나 cpu였다.

# 모델 테스트
value = torch.Tensor(1,1,28,28)
print(model(value).shape)

'\# 모델 테스트' 부분:


모델을 선언할 때는 임의의 Value를 넣어 제대로 돌아가는 모델인지 테스트를 해야 하기에 넣어 주었다.

In [0]:
# Cost Function과 Optimizer 정하기
criterion = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

In [40]:
# Training

total_batch = len(data_loader)

for epoch in range(training_epochs):
  avg_cost = 0

  for imgs, labels in data_loader:
    imgs   = imgs.to(device)
    labels = labels.to(device)

    optimizer.zero_grad()
    hypothesis = model(imgs)

    cost = criterion(hypothesis, labels)
    cost.backward()
    optimizer.step()

    avg_cost += cost / total_batch
    
  print("Epoch : {}|  average cost = {} ".format(epoch+1, avg_cost))

print("Learning Finished!")
    


Epoch : 1|  average cost = 0.24076279997825623 
Epoch : 2|  average cost = 0.0682782456278801 
Epoch : 3|  average cost = 0.05038006976246834 
Epoch : 4|  average cost = 0.04023517295718193 
Epoch : 5|  average cost = 0.0336553119122982 
Epoch : 6|  average cost = 0.028655558824539185 
Epoch : 7|  average cost = 0.024892257526516914 
Epoch : 8|  average cost = 0.02045847661793232 
Epoch : 9|  average cost = 0.018241852521896362 
Epoch : 10|  average cost = 0.014795532450079918 
Epoch : 11|  average cost = 0.01366713922470808 
Epoch : 12|  average cost = 0.011033331975340843 
Epoch : 13|  average cost = 0.009477882646024227 
Epoch : 14|  average cost = 0.009487217292189598 
Epoch : 15|  average cost = 0.007053466513752937 
Learning Finished!


In [41]:
# Test Model

with torch.no_grad():
  imgs_test = mnist_test.test_data.view(len(mnist_test), 1, 28, 28).float().to(device)
  labels_test = mnist_test.test_labels.to(device)

  prediction = model(imgs_test)
  correct_prediction = torch.argmax(prediction, 1) == labels_test
  accuracy = correct_prediction.float().mean()
  print('Accuracy: ', accuracy.item())


/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:60: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:50: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")


Accuracy:  0.9824000000953674


탐구:

 **"Layer를 더 깊게 쌓는다면 결과가 어떻게 될까?"**

 지금 이만큼 돌리는 것도 10 분 넘게, 굉장히 오래 걸려서 돌릴 엄두가 안 나긴 하는데...
 해당 코드는 [모두의딥러닝 깃헙](github/deeplearningzerotoall/PyTorch/blob/master/lab-10_2_mnist_deep_cnn.ipynb)에 올라와있다.


 강의 영상에서 알려준 결과만 보자면 Accuracy가 더 떨어진 모습을 볼 수 있었다. 따라서 모델이 깊을 수록 좋은 것이 아니고, 효율적으로 쌓는 것이 더 중요하다는 결론을 낼 수 있다.

다음 영상들에선 그 효율성을 높이기 위한 노력들을 배우겠다고 한다.
